do a two layer NN (16,10) - use vectorized math (but still try sigmoid first), do step by step

In [72]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

data = pd.read_csv('train.csv')

In [73]:
def sigmoid(x): #sigmoid func to squish all inputs into range 0 to 1
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x): #derivative of sigmoid
    return sigmoid(x) * (1 - sigmoid(x))


In [74]:
# np.outer(a, b)
'''

Given two vectors, a = [a0, a1, ..., aM] and b = [b0, b1, ..., bN], the outer product 1 is:

  [[a0*b0  a0*b1 ... a0*bN ]
   [a1*b0    .
   [ ...          .
   [aM*b0            aM*bN ]]
'''

'\n\nGiven two vectors, a = [a0, a1, ..., aM] and b = [b0, b1, ..., bN], the outer product 1 is:\n\n  [[a0*b0  a0*b1 ... a0*bN ]\n   [a1*b0    .\n   [ ...          .\n   [aM*b0            aM*bN ]]\n'

In [135]:
#Training data management
data= np.array(data)

#Train test split 80:20
test_datas = data[int(len(data)*0.8):]
train_datas = data[:int(len(data)*0.8)]

#Separating pixel data and label data
train_labels = train_datas[:,0] #label col
train_datas = (train_datas[:,1:] - np.min(train_datas[:,1:]))/(np.max(train_datas[:,1:])-np.min(train_datas[:,1:])) # pixel data, scaled to 0-1

test_labels = test_datas[:,0] #label col
test_datas = (test_datas[:,1:] - np.min(test_datas[:,1:]))/(np.max(test_datas[:,1:])-np.min(test_datas[:,1:])) # pixel data, scaled to 0-1

In [136]:
#Initialization of weights and biases

weights = [] #list to store all the weights for every layer
biases = [] #list to store all the biases for every layer

size = [16, 10] #size of each layer, first layer is input layer, last layer is output layer

train_data = train_datas 
train_label = train_labels

for i in range(len(size)): #Initialize weights for each layer
    if i == 0:
        weights.append(np.random.randn(size[0], len(train_data[0])) * np.sqrt(1/len(train_data[0])))
    else:
        weights.append(np.random.randn(size[i], size[i-1]) * np.sqrt(1/size[i-1]))

for i in range(len(size)):  #Initialize biases for each layer
    if i == 0:
        biases.append(np.zeros(size[0])) #First layer biases
    else:
        biases.append(np.zeros(size[i]))  

In [137]:
for _ in range(100): #Training loop, 100 epochs
    for i in range(1000): 
        #Forward pass

        z = weights[0] @ train_data[i] + biases[0] #First layer
        a = sigmoid(z)

        z_1 = weights[1] @ a + biases[1] #output layer
        a_1 = sigmoid(z_1)


        #loss calculation (Loss vs Cost, Loss = 1 example, cost = average)

        one_hot = np.zeros(10)
        one_hot[train_label[i]] = 1

        loss = np.sum((a_1 - one_hot)**2)

        #loss calculation (Loss vs Cost, Loss = 1 example, cost = average)

        one_hot = np.zeros(10)
        one_hot[train_label[i]] = 1

        loss = np.sum((a_1 - one_hot)**2)

        # output layer Backpropagation

        dCda_1 = 2 * (a_1 - one_hot) #Derivative of cost w.r.t output layer activation
        dCdz_1 = dCda_1 * sigmoid_derivative(z_1) #Derivative of C w.r.t output layer z (dCda_1 * da_1dz_1)

        dCdw_1 = np.outer(dCdz_1, a) #Derivative of C w.r.t weights of output layer (dCdz_1 * dz_1da_1 * da_1dw_1) #It was this simple??
        dCdb_1 = dCdz_1 #Derivative of C w.r.t biases of output layer (dCdz_1 * dz_1db_1 * db_1)
        weights[1] -= 0.01 * dCdw_1 #Update weights of output layer
        biases[1] -= 0.01 * dCdb_1 #Update biases

                #16x10   #Size 10, so need transpose
        dCda = weights[1].T @ dCdz_1 #Derivative of C w.r.t activation of first layer (dCdz_1 * dz_1da_1 * da_1da)

        # First layer Backpropagation
        dCdz = dCda * sigmoid_derivative(z) 

        dCdw = np.outer(dCdz, train_data[i])
        dCdb = dCdz
        weights[0] -= 0.01 * dCdw
        biases[0] -= 0.01 * dCdb


In [138]:
loss

0.011554974624645366

In [157]:
#Forward propagation, test a single example
m = 10 #Example to test
z = weights[0] @ train_datas[m] + biases[0] #First layer
a = sigmoid(z)

z_1 = weights[1] @ a + biases[1] #Following layers
a_1 = sigmoid(z_1)

print("\nevaluation=",a_1,"max= ",np.argmax(a_1)," label= ",train_labels[m])


evaluation= [0.02579583 0.01543922 0.01920598 0.00736364 0.00195674 0.04516352
 0.00124296 0.01011642 0.96458391 0.03115745] max=  8  label=  8


In [142]:
#Check accuracy on training set
correct = 0
k = 1000
for i in range(k):
    z = weights[0] @ train_datas[i] + biases[0] #First layer
    a = sigmoid(z)

    for j in range(len(size)-1): 
        z = weights[j+1] @ a + biases[j+1] #Following layers
        a = sigmoid(z)

    if train_labels[i] == np.argmax(a): #np.argmax(a)
        correct += 1
        
print(correct/k)

0.975


In [179]:
#Check accuracy on test set
correct = 0

for i in range(len(test_datas)):
    z = weights[0] @ test_datas[i] + biases[0] #First layer
    a = sigmoid(z)

    for j in range(len(size)-1): 
        z = weights[j+1] @ a + biases[j+1] #Following layers
        a = sigmoid(z)

    if test_labels[i] == np.argmax(a): #np.argmax(a)
        correct += 1

print(correct/len(test_datas))

0.8845238095238095


In [225]:
from PIL import Image

img = Image.open("number.png") #Replace your_image.jpg with the path to your image

In [226]:
img

In [227]:
img_array = np.array(img)
img_array = (img_array[:,:,0]).flatten()
img_array = (img_array - np.min(img_array)) / (np.max(img_array) - np.min(img_array))


z = weights[0] @ img_array + biases[0]
a = sigmoid(z)

z_1 = weights[1] @ a + biases[1]
a_1 = sigmoid(z_1)

print(np.argmax(a_1))

3
